<a href="https://colab.research.google.com/github/Okitrader/freecodecamp_agloML_crypto/blob/main/FCC_Algo_Proj01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

All Packages Needed:
pandas, numpy, matplotlib, statsmodels, pandas_datareader, datetime, yfinance, sklearn, PyPortfolioOpt

In [1]:
!pip install pandas warnings pandas_ta numpy matplotlib statsmodels pandas-datareader datetime yfinance scikit-learn PyPortfolioOpt


ERROR: Could not find a version that satisfies the requirement warnings (from versions: none)
ERROR: No matching distribution found for warnings


In [2]:
!pip install pandas_ta requests tqdm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.1/115.1 kB 3.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pandas_ta: filename=pandas_ta-0.3.14b0-py3-none-any.whl size=218907 sha256=a0f32349446d1cb19492fb0108f16ea26c288b345b415b79595a4ab6effd483a
  Stored in directory: /root/.cache/pip/wheels/69/00/ac/f7fa862c34b0e2ef320175100c233377b4c558944f12474cf0
Successfully built pandas_ta


In [3]:
from statsmodels.regression.rolling import RollingOLS
import pandas_datareader.data as web
import matplotlib.pyplot as plt
import statsmodels.api as sm
import pandas as pd
import numpy as np
import datetime as dt
import pandas_ta
import warnings
warnings.filterwarnings('ignore')


## Create a list of top Cryptos Ranked by Coinmarket Cap.

In [5]:
# Import necessary libraries
import requests
import pandas as pd
from google.colab import userdata

# Retrieve API key from Colab user data
api_key = userdata.get('CMCKey')

if api_key is None:
    print("API key not found. Please check that the secret name is correct and that the notebook has access to it.")
else:
    print("API key retrieved successfully.")

# Set the limit for the number of cryptocurrencies to retrieve
TOP_CRYPTO_LIMIT = 125  # Change this number to fetch a different number of top cryptocurrencies

# CoinMarketCap API URL and parameters
url = 'https://pro-api.coinmarketcap.com/v1/cryptocurrency/listings/latest'
parameters = {
    'start': '1',
    'limit': str(TOP_CRYPTO_LIMIT),  # Convert the limit to a string to use in the parameters
    'convert': 'USD'
}
headers = {
    'Accepts': 'application/json',
    'X-CMC_PRO_API_KEY': api_key,
}

# Make the API request
try:
    response = requests.get(url, headers=headers, params=parameters)
    response.raise_for_status()  # Will raise an HTTPError if the HTTP request returned an unsuccessful status code
    data = response.json()

    # Parse the data to extract 'cmc_rank' and 'symbol'
    cryptos = [{'cmc_rank': entry['cmc_rank'], 'symbol': entry['symbol']} for entry in data['data']]

    # Create DataFrame and name it top_cryptos (renamed to reflect the variable limit)
    top_cryptos = pd.DataFrame(cryptos)

    # Verify that the ranks are within the top limit set
    if top_cryptos['cmc_rank'].max() <= TOP_CRYPTO_LIMIT and top_cryptos['cmc_rank'].min() >= 1:
        print(f"Successfully retrieved top {TOP_CRYPTO_LIMIT} cryptocurrencies by cmc_rank.")
    else:
        print(f"Some cryptocurrencies may not be in the top {TOP_CRYPTO_LIMIT} by cmc_rank.")

    # Display the DataFrame
    print(top_cryptos.head())
    print(top_cryptos.tail())
except requests.exceptions.HTTPError as errh:
    print("Http Error:", errh)
except requests.exceptions.ConnectionError as errc:
    print("Error Connecting:", errc)
except requests.exceptions.Timeout as errt:
    print("Timeout Error:", errt)
except requests.exceptions.RequestException as err:
    print("Oops: Something Else", err)

API key retrieved successfully.
Successfully retrieved top 125 cryptocurrencies by cmc_rank.
   cmc_rank symbol
0         1    BTC
1         2    ETH
2         3   USDT
3         4    BNB
4         5    XRP
     cmc_rank symbol
120       121    CVX
121       122    SFP
122       123  TFUEL
123       124    HNT
124       125    JST


## Next we create a Dataframe for our OHLCV
we are not using adjusted close as in the video examples, hence our formulas will have to change with our version

In [7]:
from tqdm import tqdm
import pprint as pp
import datetime
pd.set_option('display.max_columns', None)
from google.colab import userdata # Ensure this contains your CoinMarketCap API key

# Retrieve API key from Colab user data
api_key = userdata.get('CMCKey')

# Function to fetch and process historical cryptocurrency data for each symbol
def fetch_historical_crypto_data(start_date=None, end_date=None, days_back=29):
    # Default to current date if end_date is not specified
    if end_date is None:
        end_date = datetime.datetime.now()

    # Default to 29 days back if start_date is not specified
    if start_date is None:
        start_date = end_date - datetime.timedelta(days=days_back)

    # Convert dates to string format if they are datetime objects
    if isinstance(start_date, datetime.datetime):
        start_date = start_date.strftime('%Y-%m-%d')
    if isinstance(end_date, datetime.datetime):
        end_date = end_date.strftime('%Y-%m-%d')

    base_url = 'https://pro-api.coinmarketcap.com/v2/cryptocurrency/ohlcv/historical'
    headers = {
        'Accepts': 'application/json',
        'X-CMC_PRO_API_KEY': api_key,
    }
    all_data = []

    # Using tqdm for progress display
    for symbol in tqdm(top_cryptos['symbol'], desc="Processing", unit="symbol"):
        url = f'{base_url}?symbol={symbol}'
        parameters = {
            'time_start': start_date,
            'time_end': end_date,
            'convert': 'USD'
        }

        try:
            response = requests.get(url, headers=headers, params=parameters)
            data = response.json()

            # Extract and flatten the quotes data
            quotes = data.get('data', {}).get(symbol, [])
            for quote in quotes:
                for q in quote.get('quotes', []):
                    usd_data = q['quote']['USD']
                    entry = {
                        'symbol': symbol,
                        'timestamp': usd_data['timestamp'],
                        'open': usd_data['open'],
                        'high': usd_data['high'],
                        'low': usd_data['low'],
                        'close': usd_data['close'],
                        'volume': usd_data['volume']
                    }
                    all_data.append(entry)

        except requests.exceptions.RequestException as e:
            print(f"Error fetching data for {symbol}: {e}")

    # Convert all_data to a DataFrame
    top_OHLCV = pd.DataFrame(all_data)

    # Set the index to a MultiIndex of 'timestamp' and 'symbol'
    top_OHLCV.set_index(['timestamp', 'symbol'], inplace=True)

    # Ensure the column names are lowercase
    top_OHLCV.columns = top_OHLCV.columns.str.lower()

    # Round the volume column to 4 decimal places
    top_OHLCV['volume'] = top_OHLCV['volume'].round(6)

    # Return the final DataFrame
    return top_OHLCV

# Fetching data
top_OHLCV = fetch_historical_crypto_data()

# Print the data using pretty print
pp.pprint(top_OHLCV.head())
pp.pprint(top_OHLCV.tail())

Processing: 100%|██████████| 125/125 [00:41<00:00,  2.99symbol/s]

                                         open          high           low  \
timestamp                symbol                                             
2023-10-14T23:59:59.999Z BTC     26866.203245  26968.999218  26814.586586   
2023-10-15T23:59:59.999Z BTC     26858.011726  27289.170319  26817.894010   
2023-10-16T23:59:59.999Z BTC     27162.628229  29448.139037  27130.473478   
2023-10-17T23:59:59.999Z BTC     28522.098166  28618.752390  28110.186117   
2023-10-18T23:59:59.999Z BTC     28413.530808  28889.009589  28174.252551   

                                        close        volume  
timestamp                symbol                              
2023-10-14T23:59:59.999Z BTC     26861.706203  5.388117e+09  
2023-10-15T23:59:59.999Z BTC     27159.652919  7.098202e+09  
2023-10-16T23:59:59.999Z BTC     28519.466679  2.783388e+10  
2023-10-17T23:59:59.999Z BTC     28415.748140  1.487253e+10  
2023-10-18T23:59:59.999Z BTC     28328.341152  1.272413e+10  
                          